# **Part-6: Stream & Tasks for Curated Layers - Enhanced**

**Objectives of part-6:**
* In Part 6, we delve into creating streams and tasks within the curated layer of the ETL process.
* The focal point is establishing the data flow between curated and consumption zones, mirroring the approach we've previously used between the landing and curated zones.

**Stream and Task Creation:**

* We embark on stream and task creation within the curated zone and consumption zone, showcasing the seamless flow of data across zones.
* We navigate through this process step by step, elucidating each action and rationale.

**Creating a Stream in Curated Zone:**

* We initiate by configuring a stream within the curated zone, ensuring a shift in context to the desired schema.
* This stream is designated for curated layer operations involving the item, customer, and order tables.
* Importantly, we highlight the distinction between append-only and delta streams, emphasizing the adaptability to different business needs.

In [ ]:
use schema chK.curated_zone;
create or replace stream curated_item_stm on table curated_item;
create or replace stream curated_customer_stm on table curated_customer;
create or replace stream curated_order_stm on table curated_order;

**Task Creation for Dimension and Fact Tables:**

* Next, we transition to task creation within the consumption zone, starting with the dimension tables (customer and item) and then the fact table (order).
* Task creation involves merging data from curated layers based on various criteria.
* A significant element in this process is the utilization of **`metadata operations as validators`**, warranting precision in data management.


In [ ]:
use schema chk.consumption_zone;

create or replace task item_consumption_tsk
  warehouse = compute_wh
  schedule  = '4 minute'
when
    system$stream_has_data('chk.curated_zone.curated_item_stm')
as
  merge into chk.consumption_zone.item_dim item using chk.curated_zone.curated_item_stm curated_item_stm on
  item.item_id = curated_item_stm.item_id and
  item.start_date = curated_item_stm.start_date and
  item.item_desc = curated_item_stm.item_desc
when matched
  and curated_item_stm.METADATA$ACTION = 'INSERT'
  and curated_item_stm.METADATA$ISUPDATE = 'TRUE'
  then update set
      item.end_date = curated_item_stm.end_date,
      item.price = curated_item_stm.price,
      item.item_class = curated_item_stm.item_class,
      item.item_category = curated_item_stm.item_category
when matched
  and curated_item_stm.METADATA$ACTION = 'DELETE'
  and curated_item_stm.METADATA$ISUPDATE = 'FALSE'
  then update set
      item.active_flag = 'N',
      updated_timestamp = current_timestamp()
when not matched
  and curated_item_stm.METADATA$ACTION = 'INSERT'
  and curated_item_stm.METADATA$ISUPDATE = 'FALSE'
then
  insert (
    item_id,
    item_desc,
    start_date,
    end_date,
    price,
    item_class,
    item_category)
  values (
    curated_item_stm.item_id,
    curated_item_stm.item_desc,
    curated_item_stm.start_date,
    curated_item_stm.end_date,
    curated_item_stm.price,
    curated_item_stm.item_class,
    curated_item_stm.item_category);



In the above code We are creating a task for the fact table. This table focuses on customer and item information, which are like categories. The item table holds important core details, and it's somewhat similar to how the customer table is set up.

But there are some small differences. We're using something called a 'metadata operation' to double-check things in this task. We call this task the 'Item Consumption Task.' It uses a tool called 'compute warehouse' to get things done, and it's set to run every four minutes. What's interesting is that this task only goes ahead if there are new changes in the item list we're keeping. If there aren't any new changes, it won't do anything.

The main job of this task is to match up data between our item list and the list we're updating. It looks at three important things: whether items have the same ID, if they started at the same time, and if their descriptions match. When all these things match and we're adding new stuff ('insert' and 'true'), the task makes updates. But if we're getting rid of things ('delete' and 'false'), it turns off the active flag, showing that something got deleted.

In [ ]:
create or replace task customer_consumption_tsk
    warehouse = compute_wh
schedule  = '5 minute'
when
  system$stream_has_data('chk.curated_zone.curated_customer_stm')
as
  merge into chk.consumption_zone.customer_dim customer using chk.curated_zone.curated_customer_stm curated_customer_stm on
  customer.customer_id = curated_customer_stm.customer_id
when matched
  and curated_customer_stm.METADATA$ACTION = 'INSERT'
  and curated_customer_stm.METADATA$ISUPDATE = 'TRUE'
  then update set
      customer.salutation = curated_customer_stm.salutation,
      customer.first_name = curated_customer_stm.first_name,
      customer.last_name = curated_customer_stm.last_name,
      customer.birth_day = curated_customer_stm.birth_day,
      customer.birth_month = curated_customer_stm.birth_month,
      customer.birth_year = curated_customer_stm.birth_year,
      customer.birth_country = curated_customer_stm.birth_country,
      customer.email_address = curated_customer_stm.email_address
when matched
  and curated_customer_stm.METADATA$ACTION = 'DELETE'
  and curated_customer_stm.METADATA$ISUPDATE = 'FALSE'
  then update set
      customer.active_flag = 'N',
      customer.updated_timestamp = current_timestamp()
when not matched
  and curated_customer_stm.METADATA$ACTION = 'INSERT'
  and curated_customer_stm.METADATA$ISUPDATE = 'FALSE'
then
  insert (
    customer_id ,
    salutation ,
    first_name ,
    last_name ,
    birth_day ,
    birth_month ,
    birth_year ,
    birth_country ,
    email_address )
  values (
    curated_customer_stm.customer_id ,
    curated_customer_stm.salutation ,
    curated_customer_stm.first_name ,
    curated_customer_stm.last_name ,
    curated_customer_stm.birth_day ,
    curated_customer_stm.birth_month ,
    curated_customer_stm.birth_year ,
    curated_customer_stm.birth_country ,
    curated_customer_stm.email_address);


Here we've created a 'customer consumption task.' This task is programmed to run every five minutes, activating the merge statement exclusively when the curated customer stream holds change data capture (CDC) information. In other words, it will only work when there are updates in the customer data.

When the task identifies a matching record based on the customer ID, and both the 'insert' and 'update' flags are true, it signifies that this particular record has been modified. Consequently, I'll proceed to update all the relevant fields in accordance with this match.

Conversely, if the 'update' flag is false and the customer remains in the dimension table while being marked as 'deleted' within the stream data, the task will take action to set the 'active' flag as 'no.' This denotes the removal of the customer from the active list.

Lastly, if there's no match found, indicating a new arriving customer, the task will initiate an 'insert' operation to add this customer to the dimension. This encapsulates how the CDC mechanism for customer dimension behaves.



In [ ]:
create or replace task order_fact_tsk
warehouse = compute_wh
schedule  = '6 minute'
when
  system$stream_has_data('chk.curated_zone.curated_order_stm')
as
insert overwrite into chk.consumption_zone.order_fact (
order_date,
customer_dim_key ,
item_dim_key ,
order_count,
order_quantity ,
sale_price ,
disount_amt ,
coupon_amt ,
net_paid ,
net_paid_tax ,
net_profit)
select
      co.order_date,
      cd.customer_dim_key ,
      id.item_dim_key,
      count(1) as order_count,
      sum(co.order_quantity) ,
      sum(co.sale_price) ,
      sum(co.disount_amt) ,
      sum(co.coupon_amt) ,
      sum(co.net_paid) ,
      sum(co.net_paid_tax) ,
      sum(co.net_profit)
  from chk.curated_zone.curated_order co
    join chk.consumption_zone.customer_dim cd on cd.customer_id = co.customer_id
    join chk.consumption_zone.item_dim id on id.item_id = co.item_id and id.item_desc = co.item_desc and id.end_date is null
    group by
        co.order_date,
        cd.customer_dim_key ,
        id.item_dim_key
        order by co.order_date;


This is the  final task, 'Order Fact Task.' Here,  for simplicity we've decided not to use any merge operations. Instead, we keeping track of changes in the order table. The reason is that we are dealing with an aggregation, making straightforward update statements unfeasible. This task is scheduled to run every six minutes, exclusively performing the aggregation we previously executed as part of a one-time load.

To elaborate, when a customer match occurs, along with a corresponding item match in terms of item ID, item description, and a non-null end date, the aggregation process is triggered. Admittedly, this isn't the most comprehensive implementation, but it serves to demonstrate the data movement from one end to another.



**Task Validation and Execution:**

* The execution of tasks is verified, showcasing successful task resumption.
* We provide a comprehensive look into the execution history of tasks, highlighting scheduled times and executed statements.

In [ ]:
alter task order_curated_tsk resume;
alter task customer_curated_tsk resume;
alter task item_curated_tsk resume;

select *  from table(information_schema.task_history())
where name in ('CUSTOMER_CURATED_TSK' ,'ITEM_CURATED_TSK','ORDER_CURATED_TSK')
order by scheduled_time;

**Completing Part 6:**

* As we wrap up Part 6, we reflect on the comprehensive process we've undertaken:
* Creation of streams and tasks in all zones.
* Successful execution and resumption of tasks.
* Functioning streams transferring data across zones.
* A total of around 24 to 25 objects established within the ETL process.

The completion of Part 6 sets the stage for the subsequent phase, where we will explore the data loading process. Our journey continues as we move from the point of data ingestion to its transition through various zones, ultimately reaching the consumption zone.